In [277]:
import pandas as pd
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline 

# 60mins abs difference

In [278]:
#for getting the price change (price before tweet and price after the tweet) #5mins interval one
df2 = pd.read_csv("tweets_stocks_combined_final_60mins.csv")

In [279]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      2246 non-null   float64
 1   text                    2198 non-null   object 
 2   favorites               2246 non-null   int64  
 3   retweets                2246 non-null   int64  
 4   date                    2246 non-null   object 
 5   tweet_datetime          2246 non-null   object 
 6   date_part               2246 non-null   object 
 7   time_part               2246 non-null   object 
 8   hour                    2246 non-null   int64  
 9   year                    2246 non-null   int64  
 10  month                   2246 non-null   int64  
 11  topic                   2246 non-null   int64  
 12  compound                2246 non-null   float64
 13  datetime_60mins_after   2246 non-null   object 
 14  price_60mins_after      2246 non-null   

In [280]:
df2 = df2[df2['text'].notnull()]

In [281]:
a = df2.loc[:,"60mins_price_diff_abs"]
a.describe(percentiles = [0.40, 0.60])

count    2198.000000
mean        0.005884
std         0.432532
min        -3.990000
40%        -0.003393
50%         0.003750
60%         0.020000
max         2.970000
Name: 60mins_price_diff_abs, dtype: float64

In [282]:
df2

,id,text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,month,topic,compound,datetime_60mins_after,price_60mins_after,price_now,60mins_price_diff_abs,60mins_price_diff_perc
0,9.353400e+17,thank rand,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,1,0.4199,2017-11-28 11:50:00,261.485000,261.100000,0.385000,0.001475
1,8.997980e+17,join live fort myer arlington virginia,36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,2,0.2960,2017-08-22 10:00:00,244.260000,243.670000,0.590000,0.002421
2,8.939700e+17,thank nicole,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,5,6,0.4199,2017-05-09 08:01:00,239.940000,239.875000,0.065000,0.000271
3,8.819770e+17,thank shawn steel nice word,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,3,3,0.6486,2017-03-08 05:44:00,237.022857,236.880000,0.142857,0.000603
4,8.787250e+17,make america great,134210,36346,2017-06-24 21:23:00,2017-06-25 05:23:00,2017-06-25,05:23:00,5,2017,6,2,0.6588,2017-06-25 06:23:00,243.326476,243.320762,0.005714,0.000023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,9.990960e+17,person placed early campaign spy put previous ...,78529,20098,2018-05-23 01:13:00,2018-05-23 09:13:00,2018-05-23,09:13:00,9,2018,5,3,0.6792,2018-05-23 10:13:00,271.930000,271.040000,0.890000,0.003284
2242,9.874600e+17,general michael flynn life totally destroyed s...,93569,25259,2018-04-20 10:34:00,2018-04-20 18:34:00,2018-04-20,18:34:00,18,2018,4,3,-0.6689,2018-04-20 19:34:00,267.025000,266.820000,0.205000,0.000768
2243,9.870960e+17,thought prayer condolence family friend collea...,62645,16081,2018-04-19 22:30:00,2018-04-20 06:30:00,2018-04-20,06:30:00,6,2018,4,2,0.6900,2018-04-20 07:30:00,269.070000,268.620000,0.450000,0.001675
2244,9.863570e+17,today court decision mean congress must close ...,56749,12426,2018-04-17 21:34:00,2018-04-18 05:34:00,2018-04-18,05:34:00,5,2018,4,2,-0.9493,2018-04-18 06:34:00,270.695000,270.600000,0.095000,0.000351


In [283]:
#creates a list of values for the y label for each row
target_value = []

for i in range(0, len(df2)):
    
    if df2.iloc[i, 16] >= 0.020000:
        target_value.append(1)

    elif df2.iloc[i, 16] <= -0.003393:
        target_value.append(-1)

    else:
        target_value.append(0)
    

In [284]:
compound_group = []

for i in range(0, len(df2)):
    
    if df2.iloc[i, 12] >= 0.6:
        compound_group.append(15)
        
    elif df2.iloc[i, 12] >= 0.1:
        compound_group.append(14)
      
    elif df2.iloc[i, 12] > -0.1 and df2.iloc[i, 12] < 0.1:
        compound_group.append(13)
        
    elif df2.iloc[i, 12] <= -0.6:
        compound_group.append(11)
    
    elif df2.iloc[i, 12] <= -0.1:
        compound_group.append(12)

In [285]:
hours_grouped = []

for i in range(0, len(df2)):
    
    if df2.iloc[i, 8] >= 16:
        hours_grouped.append(20)
        
    elif df2.iloc[i, 8] >= 12:
        hours_grouped.append(19)
        
    else:
        hours_grouped.append(18)

In [286]:
#create a column for the y_label 
df2.loc[:, 'hours_grouped'] = hours_grouped
df2.loc[:, 'target'] = target_value
df2.loc[:, 'compound_group'] = compound_group

In [287]:
#to verify that there are no NA values
df2.isna().sum()

id                        0
text                      0
favorites                 0
retweets                  0
date                      0
tweet_datetime            0
date_part                 0
time_part                 0
hour                      0
year                      0
month                     0
topic                     0
compound                  0
datetime_60mins_after     0
price_60mins_after        0
price_now                 0
60mins_price_diff_abs     0
60mins_price_diff_perc    0
hours_grouped             0
target                    0
compound_group            0
dtype: int64

In [288]:
df2 = df2[['topic', 'compound_group', 'hours_grouped', 'target']]

comp = pd.get_dummies(df2['compound_group'])
topic = pd.get_dummies(df2['topic'])
hrs = pd.get_dummies(df2['hours_grouped'])
# Drop column as it is now encoded
df2 = df2.drop(['compound_group'], axis = 1)
df2 = df2.drop(['topic'], axis = 1)
df2 = df2.drop(['hours_grouped'], axis = 1)
# Join the encoded df
df2 = df2.join(comp)
df2 = df2.join(topic)
df2 = df2.join(hrs)
df2

,target,11,12,13,14,15,0,1,2,3,4,5,6,7,8,9,18,19,20
0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
3,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
2242,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2243,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
2244,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0


In [289]:
X = df2.iloc[:, 1:]
y = df2.target

In [290]:
#Train-test split
#test size is 20% of the dataset
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [291]:
smote = SMOTE(random_state=0, sampling_strategy='not majority')
x_train, y_train = smote.fit_sample(x_train, y_train)

In [292]:
y_train.value_counts()

 1    710
-1    710
 0    710
Name: target, dtype: int64

In [293]:
y_test.value_counts()

 1    179
-1    167
 0     94
Name: target, dtype: int64

# With SMOTE but without Hyperparameter tuning

In [294]:
clf = MultinomialNB()
model = clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Accuracy of Naive Bayes Classifier: {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print("\nConfusion Matrix of Naive Bayes Classifier:\n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report of Naive Bayes Classifier:\n")
print(classification_report(y_test, y_pred))

Accuracy of Naive Bayes Classifier: 34.55%

Confusion Matrix of Naive Bayes Classifier:

[[116  26  25]
 [ 62  17  15]
 [130  30  19]]

Classification Report of Naive Bayes Classifier:

              precision    recall  f1-score   support

          -1       0.38      0.69      0.49       167
           0       0.23      0.18      0.20        94
           1       0.32      0.11      0.16       179

    accuracy                           0.35       440
   macro avg       0.31      0.33      0.28       440
weighted avg       0.32      0.35      0.29       440



In [295]:
logr = LogisticRegression()
model = logr.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Accuracy of Logistic Regression Classifier: {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print("\nConfusion Matrix of Logistic Regression Classifier:\n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report of Logistic Regression Classifier:\n")
print(classification_report(y_test, y_pred))

Accuracy of Logistic Regression Classifier: 38.18%

Confusion Matrix of Logistic Regression Classifier:

[[76 28 63]
 [34 22 38]
 [74 35 70]]

Classification Report of Logistic Regression Classifier:

              precision    recall  f1-score   support

          -1       0.41      0.46      0.43       167
           0       0.26      0.23      0.25        94
           1       0.41      0.39      0.40       179

    accuracy                           0.38       440
   macro avg       0.36      0.36      0.36       440
weighted avg       0.38      0.38      0.38       440



C:\Users\vedan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [296]:
clf = SVC()
model = clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Accuracy of SVC Classifier: {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print("\nConfusion Matrix of SVC Classifier:\n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report of SVC Classifier:\n")
print(classification_report(y_test, y_pred))

Accuracy of SVC Classifier: 37.95%

Confusion Matrix of SVC Classifier:

[[167   0   0]
 [ 94   0   0]
 [179   0   0]]

Classification Report of SVC Classifier:

              precision    recall  f1-score   support

          -1       0.38      1.00      0.55       167
           0       0.00      0.00      0.00        94
           1       0.00      0.00      0.00       179

    accuracy                           0.38       440
   macro avg       0.13      0.33      0.18       440
weighted avg       0.14      0.38      0.21       440



C:\Users\vedan\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [297]:
clf = RandomForestClassifier()
model = clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Accuracy of Random Forest Classifier: {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print("\nConfusion Matrix of Random Forest Classifier:\n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report of Random Forest Classifier:\n")
print(classification_report(y_test, y_pred))

Accuracy of Random Forest Classifier: 34.09%

Confusion Matrix of Random Forest Classifier:

[[52 59 56]
 [22 26 46]
 [45 62 72]]

Classification Report of Random Forest Classifier:

              precision    recall  f1-score   support

          -1       0.44      0.31      0.36       167
           0       0.18      0.28      0.22        94
           1       0.41      0.40      0.41       179

    accuracy                           0.34       440
   macro avg       0.34      0.33      0.33       440
weighted avg       0.37      0.34      0.35       440



In [298]:
classifier = LGBMClassifier(objective='multiclass', random_state=0, n_jobs=-1)
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

print("Accuracy of Light Gradient Boosting Method Classifier: {}%".format(round(accuracy_score(y_test, y_pred)*100,2)))
print("\nConfusion Matrix of Light Gradient Boosting Method Classifier:\n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report of Light Gradient Boosting Method Classifier:\n")
print(classification_report(y_test, y_pred))

Accuracy of Light Gradient Boosting Method Classifier: 34.32%

Confusion Matrix of Light Gradient Boosting Method Classifier:

[[50 65 52]
 [22 34 38]
 [40 72 67]]

Classification Report of Light Gradient Boosting Method Classifier:

              precision    recall  f1-score   support

          -1       0.45      0.30      0.36       167
           0       0.20      0.36      0.26        94
           1       0.43      0.37      0.40       179

    accuracy                           0.34       440
   macro avg       0.36      0.35      0.34       440
weighted avg       0.39      0.34      0.35       440



# With SMOTE and Hyperparameter tuning

In [299]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'alpha' : [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
}
# Create a based model
nb = MultinomialNB()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = nb, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, scoring = 'f1_weighted')

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)
print()

best_grid = grid_search.best_estimator_

def evaluate(model, test_features, test_labels):
    y_pred = model.predict(test_features)
    accuracy = accuracy_score(y_test, y_pred)
    print (accuracy * 100)
    print()
    print(confusion_matrix(y_test,y_pred))
    print()
    print(classification_report(y_test, y_pred))

evaluate(best_grid,x_test,y_test)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'alpha': 1e-05}

34.54545454545455

[[116  26  25]
 [ 62  17  15]
 [130  30  19]]

              precision    recall  f1-score   support

          -1       0.38      0.69      0.49       167
           0       0.23      0.18      0.20        94
           1       0.32      0.11      0.16       179

    accuracy                           0.35       440
   macro avg       0.31      0.33      0.28       440
weighted avg       0.32      0.35      0.29       440



[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:    2.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    2.1s finished


In [300]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs'],
    'max_iter': [500, 1000],
    'multi_class': ['auto', 'multinomial']
}
# Create a based model
lr = LogisticRegression()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = lr, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, scoring = 'f1_weighted')

# Fit the grid search to the data
grid_search.fit(x_train,y_train)

print(grid_search.best_params_)
print()

best_grid = grid_search.best_estimator_

evaluate(best_grid,x_test,y_test)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 112 candidates, totalling 336 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 321 out of 336 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 336 out of 336 | elapsed:    2.6s finished


{'C': 100, 'max_iter': 500, 'multi_class': 'auto', 'penalty': 'l2', 'solver': 'lbfgs'}

36.13636363636364

[[58 46 63]
 [25 31 38]
 [60 49 70]]

              precision    recall  f1-score   support

          -1       0.41      0.35      0.37       167
           0       0.25      0.33      0.28        94
           1       0.41      0.39      0.40       179

    accuracy                           0.36       440
   macro avg       0.35      0.36      0.35       440
weighted avg       0.37      0.36      0.36       440



In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10,15],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5, 6],
    'min_samples_split': [3, 4, 5, 6],
    'n_estimators': [1000, 1300]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, scoring = 'f1_weighted')

# Fit the grid search to the data
grid_search.fit(x_train,y_train)

print(grid_search.best_params_)

best_grid = grid_search.best_estimator_

evaluate(best_grid,x_test,y_test)

Fitting 3 folds for each of 128 candidates, totalling 384 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.9s


In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}
# Create a based model
svc = SVC()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = svc, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, scoring = 'f1_weighted')

# Fit the grid search to the data
grid_search.fit(x_train,y_train)

print(grid_search.best_params_)

best_grid = grid_search.best_estimator_

evaluate(best_grid,x_test,y_test)

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'num_leaves': [31, 127],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [10, 20, 30, 50, 100, 300, 400],
    'lambda_l1': [0.1, 1, 1.5],
    'lambda_l2': [0.0, 1]
}
# Create a based model
lgbm = LGBMClassifier(boosting_type='gbdt',  objective='multiclass', random_state=0, n_jobs=-1)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = lgbm, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, scoring = 'f1_weighted')

# Fit the grid search to the data
grid_search.fit(x_train,y_train)

print(grid_search.best_params_)

best_grid = grid_search.best_estimator_

evaluate(best_grid,x_test,y_test)